In [ ]:
# Load Chan-Lam dataset
chan_lam_path = "ord-data/data/5c/ord_dataset-5c9a10329a8a48968d18879a48bb8ab2.pb.gz"
chan_lam = message_helpers.load_message(chan_lam_path, dataset_pb2.Dataset)
print(f"Dataset: {chan_lam.name}")
print(f"Reactions: {len(chan_lam.reactions)}")

# Pick first reaction
rxn = chan_lam.reactions[0]

print("\n=== INPUTS (Reactants) ===")
for key, inp in rxn.inputs.items():
    print(f"\n{key}:")
    for comp in inp.components:
        role = reaction_pb2.ReactionRole.ReactionRoleType.Name(comp.reaction_role)
        for ident in comp.identifiers:
            if reaction_pb2.CompoundIdentifier.CompoundIdentifierType.Name(ident.type) == "SMILES":
                print(f"  {role}: {ident.value}")

print("\n=== OUTCOMES (Products) ===")
for outcome in rxn.outcomes:
    for product in outcome.products:
        for ident in product.identifiers:
            if reaction_pb2.CompoundIdentifier.CompoundIdentifierType.Name(ident.type) == "SMILES":
                print(f"  PRODUCT: {ident.value}")

NameError: name 'message_helpers' is not defined